In [1]:
import numpy as np

#############################
# MDP Definition
#############################
# States: s1, s2, s3 (we’ll index them as 0, 1, 2)
# Terminal state: s3 (index 2) with v(s3)=0.
# Actions available in s1 and s2: a1 and a2.
#
# Transitions and rewards:
# For s1 (index 0):
#   - a1: deterministically go to s2 (index 1) with reward 2.
#   - a2: deterministically go to s3 (index 2) with reward 5.
#
# For s2 (index 1):
#   - a1: deterministically go to s3 (index 2) with reward 1.
#   - a2: deterministically go to s1 (index 0) with reward -1.
#
# Discount factor
gamma = 0.9

######################################
# Q2: Policy Evaluation for a given π
######################################
# Given deterministic policy π:
#   π(s1) = a1
#   π(s2) = a1
#
# Under this policy, only the following transitions occur:
#   s1 --[a1]--> s2 with reward 2, and
#   s2 --[a1]--> s3 with reward 1.
#
# (a) Analytical solution using matrix inversion:
#
# For nonterminal states s1 and s2, the Bellman equations are:
#   v(s1) = 2 + gamma * v(s2)
#   v(s2) = 1 + gamma * v(s3) = 1   (since v(s3)=0)
#
# In matrix form (for v = [v(s1), v(s2)]):
#   [1    -gamma] [v(s1)]   = [2]
#   [0      1   ] [v(s2)]     [1]
A = np.array([[1, -gamma],
              [0, 1]])
b = np.array([2, 1])
v_policy_analytical = np.linalg.solve(A, b)
v1_pi_analytical = v_policy_analytical[0]
v2_pi_analytical = v_policy_analytical[1]
v3_pi_analytical = 0  # Terminal

# (b) Numerical solution by iterative updates:
v_policy_iter = np.zeros(3)  # [v(s1), v(s2), v(s3)]
tol = 1e-6
while True:
    v_new = np.copy(v_policy_iter)
    # For s1: only action a1 is taken, leading to s2.
    v_new[0] = 2 + gamma * v_policy_iter[1]
    # For s2: under a1, it goes to terminal s3.
    v_new[1] = 1  # since 1 + gamma * 0 = 1
    # s3 remains terminal with value 0.
    if np.max(np.abs(v_new - v_policy_iter)) < tol:
        break
    v_policy_iter = v_new

v1_pi_iter, v2_pi_iter, v3_pi_iter = v_policy_iter

print("=== Q2: Policy Evaluation (π(s1)=a1, π(s2)=a1) ===")
print("Analytical solution:")
print("  v(s1) = {:.4f}, v(s2) = {:.4f}, v(s3) = 0".format(v1_pi_analytical, v2_pi_analytical))
print("Iterative solution:")
print("  v(s1) = {:.4f}, v(s2) = {:.4f}, v(s3) = 0".format(v1_pi_iter, v2_pi_iter))


##############################################
# Q3: Finding the Optimal Policy and Value v*
##############################################
#
# The Bellman optimality equations for each nonterminal state are:
#
# For s1:
#   v*(s1) = max { 
#         a1: 2 + gamma*v*(s2), 
#         a2: 5 + gamma*v*(s3) } = max { 2 + 0.9*v*(s2), 5 }
#
# For s2:
#   v*(s2) = max {
#         a1: 1 + gamma*v*(s3), 
#         a2: -1 + gamma*v*(s1) } = max { 1, -1 + 0.9*v*(s1) }
#
# And v*(s3)=0.
#
# (a) Analytical solution:
# We need to choose the actions that maximize the right-hand sides.
# Let’s denote:
#   x = v*(s1) and y = v*(s2)
#
# Suppose the optimal action in s1 is a1 and in s2 is a2.
# Then:
#   x = 2 + 0.9*y
#   y = -1 + 0.9*x
#
# Substitute y from the second into the first:
#   x = 2 + 0.9*(-1 + 0.9*x) = 2 - 0.9 + 0.81*x
#   => x - 0.81*x = 1.1   => 0.19*x = 1.1   => x ≈ 5.7895
# Then:
#   y = -1 + 0.9*5.7895 ≈ -1 + 5.21055 ≈ 4.21055
#
# We verify the choices:
# For s1: a1 yields 2+0.9*y ≈ 2+3.7895=5.7895; a2 yields 5.
# Hence, a1 is optimal since 5.7895 > 5.
#
# For s2: a1 yields 1; a2 yields -1+0.9*x ≈ -1+5.21055=4.21055.
# Hence, a2 is optimal since 4.21055 > 1.
#
v1_opt_analytical = 5.7895
v2_opt_analytical = 4.21055
v3_opt_analytical = 0

print("\n=== Q3: Optimal Value Function (Analytical) ===")
print("Optimal value function:")
print("  v*(s1) = {:.5f}, v*(s2) = {:.5f}, v*(s3) = 0".format(v1_opt_analytical, v2_opt_analytical))
print("Optimal policy (analytical):")
print("  π*(s1) = a1   (since 2+0.9*v*(s2) ≈ 5.7895 > 5)")
print("  π*(s2) = a2   (since -1+0.9*v*(s1) ≈ 4.21055 > 1)")

# (b) Numerical solution via value iteration:
v_opt = np.zeros(3)
while True:
    v_new = np.copy(v_opt)
    # Terminal state remains 0:
    v_new[2] = 0
    # For s1:
    #   a1: 2 + gamma*v(s2)
    #   a2: 5 + gamma*v(s3) = 5
    val_a1 = 2 + gamma * v_opt[1]
    val_a2 = 5
    v_new[0] = max(val_a1, val_a2)
    # For s2:
    #   a1: 1 + gamma*v(s3) = 1
    #   a2: -1 + gamma*v(s1)
    val_a1_s2 = 1
    val_a2_s2 = -1 + gamma * v_opt[0]
    v_new[1] = max(val_a1_s2, val_a2_s2)
    if np.max(np.abs(v_new - v_opt)) < tol:
        break
    v_opt = v_new

v1_opt_vi, v2_opt_vi, v3_opt_vi = v_opt

# Derive the optimal policy from the converged value function:
# For s1:
if (2 + gamma * v_opt[1]) >= 5:
    policy_s1 = 'a1'
else:
    policy_s1 = 'a2'
# For s2:
if (1) >= (-1 + gamma * v_opt[0]):
    policy_s2 = 'a1'
else:
    policy_s2 = 'a2'

print("\n=== Q3: Optimal Value Function (Value Iteration) ===")
print("Optimal value function from value iteration:")
print("  v*(s1) = {:.5f}, v*(s2) = {:.5f}, v*(s3) = 0".format(v1_opt_vi, v2_opt_vi))
print("Optimal policy from value iteration:")
print("  π*(s1) = {}".format(policy_s1))
print("  π*(s2) = {}".format(policy_s2))


=== Q2: Policy Evaluation (π(s1)=a1, π(s2)=a1) ===
Analytical solution:
  v(s1) = 2.9000, v(s2) = 1.0000, v(s3) = 0
Iterative solution:
  v(s1) = 2.9000, v(s2) = 1.0000, v(s3) = 0

=== Q3: Optimal Value Function (Analytical) ===
Optimal value function:
  v*(s1) = 5.78950, v*(s2) = 4.21055, v*(s3) = 0
Optimal policy (analytical):
  π*(s1) = a1   (since 2+0.9*v*(s2) ≈ 5.7895 > 5)
  π*(s2) = a2   (since -1+0.9*v*(s1) ≈ 4.21055 > 1)

=== Q3: Optimal Value Function (Value Iteration) ===
Optimal value function from value iteration:
  v*(s1) = 5.78947, v*(s2) = 4.21052, v*(s3) = 0
Optimal policy from value iteration:
  π*(s1) = a1
  π*(s2) = a2
